###### Importing the necessary module

In [2]:
import pandas as pd
import re
import string

##### reading the files


In [3]:
scraped= pd.read_csv('Hotel Location Table Mapping.csv',encoding='latin1')
hotels=pd.read_csv('Hotel Location.csv',encoding='latin1')
scraped_ngrams=pd.read_csv('submission.csv',encoding='latin1')

old=pd.read_csv('old.csv',encoding='latin1')
pivot=pd.read_csv('pivot.csv',encoding='latin1')

In [9]:
scraped.sample(2)

,scraped_url,agency_slug,country,title,description,inclusions,hotel_name,hotel_mapped
3233,https://www.holidayfactory.co.za/package/12868,holidayfactory,maldives,THE SUN SIYAM IRU FUSHI - 7 NIGHTS,25% discount included \nDiscounted seaplane tr...,\n\n\nReturn flights to the Maldives on Qatar\...,sun siyam iru fushi,NaN
6011,https://www.pentravel.co.za/packages/cruise-th...,pentravel,spain,Cruise the Canary Islands onboard NCL,NaN,"<ul class=""checklist""> <li>10 Nights accommoda...",NaN,NaN


The scraped data frame contains the scraped data from different agencies' websites.it contains seven columns and  8381 entries.The goalis to fill the hotel_mapped field which only contains null values,  with the a hotel name mapped from the hotels dataframe.The name should first be extracted from either the title,description or the inclusions column and then mapped to the right hotel name in the hotels table.

We have also been provided with a *hotel_name* column which also contains extracted hotel names but has only 4524 non null values,leaving 3857 null values in the hotel_name column.

In [11]:
hotels.sample(2)

,Name,Address1,Address2,City,StateProvince,PostalCode,Country,Latitude,Longitude,AirportCode,PropertyCurrency,StarRating,Location
96348,Mystic Marriott Hotel and Spa,625 North Rd,NaN,Groton,CT,6340,US,41.36485,-72.02673,PVD,USD,4.0,Near Bluff Point State Park
26542,Comfort Hotel Ulm Blaustein,Ulmer Strasse 4/1,NaN,Blaustein,NaN,89134,DE,48.42171,9.90652,STR,EUR,3.5,Near a train station


The hotels table contains the correct names and locations of the hotels

In [14]:
def clean_text(text):
    """
    This function uses regular expressions to 
        - remove links characters
        - remove html characters,
        - remove escape sequences,
        - remove capitalization,
        - any extra white space from each text and then converts them to lowercase.

    Input:
    text: original text
          datatype: string

    Output:
    texts: modified text
           datatype: string
    """
    # replace links with url-web
    pattern_url = 'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
    subs_url = 'url-web'
    text = re.sub(pattern_url, subs_url, text)
    # replace the html characters with " "
    text = re.sub('<.*?>#()&', ' ', text)
    text = re.sub('&', ' ', text)
#     text = re.sub('(', ' ', text)
#     text = re.sub(')', ' ', text)
    
    text=re.sub('Nan', ' ', text)
    # replace escape sequence with space
    escape_seq = ["\r", "\n",'\a', '\b', '\f', "\`", '\”', '\t']
    for i in escape_seq:
        text = re.sub(i, " ", text)
    # Remove Capitalization
    text = text.lower()
    # plit and join the words
    text=' '.join(text.split())
    
    return text

In [15]:
def clean(text):
    
    # removing paragraph numbers
    text = re.sub('[0-9]+.\t','',str(text))
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing hyphens
    text = re.sub("#",' ',str(text))
    text = re.sub("@",'',str(text))
    
    # removing quotation marks
    text = re.sub('\"','',str(text))
    # removing salutations
    text = re.sub("Mr\.",'Mr',str(text))
    text = re.sub("Mrs\.",'Mrs',str(text))
    # removing any reference to outside text
    text = re.sub("[\(\[].*?[\)\]]#", "", str(text))
    
    return text

The first step was to select only the rows that contained null values in the hotel_name column to easen the task.

In [12]:
selected_rows = scraped[scraped['hotel_name'].isnull()]

In [13]:
display(selected_rows.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3857 entries, 0 to 8380
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   scraped_url   3855 non-null   object 
 1   agency_slug   3855 non-null   object 
 2   country       3557 non-null   object 
 3   title         3855 non-null   object 
 4   description   2193 non-null   object 
 5   inclusions    2486 non-null   object 
 6   hotel_name    0 non-null      object 
 7   hotel_mapped  0 non-null      float64
dtypes: float64(1), object(7)
memory usage: 271.2+ KB


None

In [16]:
display(scraped.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8381 entries, 0 to 8380
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   scraped_url   8379 non-null   object 
 1   agency_slug   8379 non-null   object 
 2   country       7992 non-null   object 
 3   title         8379 non-null   object 
 4   description   5270 non-null   object 
 5   inclusions    6884 non-null   object 
 6   hotel_name    4524 non-null   object 
 7   hotel_mapped  0 non-null      float64
dtypes: float64(1), object(7)
memory usage: 523.9+ KB


None

In [17]:
nonselected_rows = scraped[scraped['hotel_name'].notnull()]
nonselected_rows['hotels_new'] = nonselected_rows['hotel_name']

C:\Users\user\AppData\Local\Temp\ipykernel_22012\1967922270.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonselected_rows['hotels_new'] = nonselected_rows['hotel_name']


In [ ]:
display(nonselected_rows.info())

In [18]:
def remove_punctuation(post):
    return ''.join([l for l in post if l not in string.punctuation])

In [19]:
scraped['description'] = scraped['description'].astype(str).apply(clean_text)
scraped['description'] = scraped['description'].astype(str).apply(remove_punctuation)

In [20]:
def domain_name(url):
    return url.split("www.")[-1].split("//")[-1].split(".")[0]

In [21]:
selected_rows['domain'] =selected_rows['scraped_url'].astype(str).apply(domain_name)

C:\Users\user\AppData\Local\Temp\ipykernel_22012\3696863612.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rows['domain'] =selected_rows['scraped_url'].astype(str).apply(domain_name)


In [22]:
type_labels = list(selected_rows.domain.unique())

In [23]:
neew = selected_rows.loc[selected_rows['domain']=='igotravel']

In [24]:
neew.sample(10)

,scraped_url,agency_slug,country,title,description,inclusions,hotel_name,hotel_mapped,domain
409,https://www.igotravel.co.za/holidays/kandima-m...,igotravel,maldives,Kandima Maldives,"<div class=""woocommerce-product-details__short...","<div class=""blck col-sm-6"">\n<h3 class=""blck-t...",NaN,NaN,igotravel
305,https://www.igotravel.co.za/holidays/mont-roch...,igotravel,south africa,"Mont Rochelle Vineyard Hotel & Spa, Western Cape","<div class=""woocommerce-product-details__short...","<div class=""blck col-sm-6"">\n<h3 class=""blck-t...",NaN,NaN,igotravel
466,https://www.igotravel.co.za/holidays/cathedral...,igotravel,united states,Cathedral Peak Hotel  Drakensberg,"<div class=""woocommerce-product-details__short...","<div class=""blck col-sm-6"">\n<h3 class=""blck-t...",NaN,NaN,igotravel
598,https://www.igotravel.co.za/holidays/victoria-...,igotravel,zambia,Victoria Falls Safari Lodge,"<div class=""woocommerce-product-details__short...","<div class=""blck col-sm-6"">\n<h3 class=""blck-t...",NaN,NaN,igotravel
566,https://www.igotravel.co.za/holidays/egyptian-...,igotravel,egypt,Egyptian Express for families,"<div class=""woocommerce-product-details__short...","<div class=""blck col-sm-6"">\n<h3 class=""blck-t...",NaN,NaN,igotravel
313,https://www.igotravel.co.za/holidays/club-med-...,igotravel,seychelles,Club Med Seychelles,"<div class=""woocommerce-product-details__short...","<div class=""blck col-sm-6"">\n<h3 class=""blck-t...",NaN,NaN,igotravel
602,https://www.igotravel.co.za/holidays/royal-liv...,igotravel,zambia,Royal Livingstone Hotel,"<div class=""woocommerce-product-details__short...","<div class=""blck col-sm-6"">\n<h3 class=""blck-t...",NaN,NaN,igotravel
468,https://www.igotravel.co.za/holidays/head-over...,igotravel,south africa,"Head over Hills, Knysna with Pezula Golf","<div class=""woocommerce-product-details__short...","<div class=""blck col-sm-6"">\n<h3 class=""blck-t...",NaN,NaN,igotravel
528,https://www.igotravel.co.za/holidays/the-taj-c...,igotravel,south africa,"The Taj, Cape Town Family Getaway","<div class=""woocommerce-product-details__short...","<div class=""blck col-sm-6"">\n<h3 class=""blck-t...",NaN,NaN,igotravel
246,https://www.igotravel.co.za/holidays/shandrani...,igotravel,mauritius,"Shandrani Hotel Honeymoon, Mauritius","<div class=""woocommerce-product-details__short...","<div class=""blck col-sm-6"">\n<h3 class=""blck-t...",NaN,NaN,igotravel


In [25]:
neew['hotels_new'] = neew['title']

C:\Users\user\AppData\Local\Temp\ipykernel_22012\1640489448.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neew['hotels_new'] = neew['title']


In [26]:
neew1 = selected_rows.loc[selected_rows['domain']=='packages']

In [27]:
neew1['hotels_new'] = neew1['title']

C:\Users\user\AppData\Local\Temp\ipykernel_22012\2116465824.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neew1['hotels_new'] = neew1['title']


In [28]:
neew2 = selected_rows.loc[selected_rows['domain']=='computravel']

In [29]:
neew2.sample(10)

,scraped_url,agency_slug,country,title,description,inclusions,hotel_name,hotel_mapped,domain
6805,https://computravel.co.za/packages/11132/zanzi...,computravel,tanzania,Zanzibar - 4* My Blue Hotel - All Inclusive - ...,"<div class=""item-description-text"">\n\n<h6>Qui...",NaN,NaN,NaN,computravel
6243,https://computravel.co.za/packages/7437/namibi...,computravel,namibia,Namibia - 4* Strand Hotel - Swakopmund - 3 Nig...,"<div class=""item-description-text"">\n\n<h6>Qui...",NaN,NaN,NaN,computravel
5678,https://computravel.co.za/packages/9458/maurit...,computravel,mauritius,Mauritius - 5* Heritage Le Telfair - Valid: 1 ...,"<div class=""item-description-text"">\n\n<h6>Qui...",NaN,NaN,NaN,computravel
4927,https://computravel.co.za/packages/10040/zanzi...,computravel,tanzania,Zanzibar - 4* Sandies Baobab - All Inclusive -...,"<div class=""item-description-text"">\n\n<h6>Qui...",NaN,NaN,NaN,computravel
5139,https://computravel.co.za/packages/10863/maldi...,computravel,maldives,Maldives - 5* Heritance Aarah - All Inclusive ...,"<div class=""item-description-text"">\n\n<h6>Qui...",NaN,NaN,NaN,computravel
4287,https://computravel.co.za/packages/10589/kwazu...,computravel,south africa,Kwazulu Natal - 4* Thonga Beach Lodge - iSiman...,"<div class=""item-description-text"">\n\n<h6>Qui...",NaN,NaN,NaN,computravel
5348,https://computravel.co.za/packages/8622/kzn-do...,computravel,south africa,KZN Dolphin Coast - 5* The Capital Zimbali - V...,"<div class=""item-description-text"">\n\n<h6>Qui...",NaN,NaN,NaN,computravel
4477,https://computravel.co.za/packages/11047/zanzi...,computravel,tanzania,Zanzibar - 5* Gold Zanzibar - All inclusive - ...,"<div class=""item-description-text"">\n\n<h6>Qui...",NaN,NaN,NaN,computravel
3902,https://computravel.co.za/packages/11057/zanzi...,computravel,tanzania,Zanzibar - 4* Sandies Baobab - All Inclusive -...,"<div class=""item-description-text"">\n\n<h6>Qui...",NaN,NaN,NaN,computravel
5555,https://computravel.co.za/packages/11165/marve...,computravel,greece,Marvels of Greece & Italy 10 Night Voyage on B...,"<div class=""item-description-text"">\n\n<h6>Qui...",NaN,NaN,NaN,computravel


In [30]:
def findAfter (txt):
#len(txt.split(''))
    myList = txt.split(" ")
    for x in myList:
        if x=='4*':
            start = txt.find (x) #Sets start to the start of keyword
            end = start + len (x) #Sets end to the end of the keyword
            return txt [end:end+30] 
        elif x=='3*':
            start = txt.find (x) #Sets start to the start of keyword
            end = start + len (x) #Sets end to the end of the keyword
            return txt [end:end+30] 
        elif x=='2*':
            start = txt.find (x) #Sets start to the start of keyword
            end = start + len (x) #Sets end to the end of the keyword
            return txt [end:end+30] 
        elif x=='5*':
            start = txt.find (x) #Sets start to the start of keyword
            end = start + len (x) #Sets end to the end of the keyword
            return txt [end:end+30] 
        elif x=='5star':
            start = txt.find (x) #Sets start to the start of keyword
            end = start + len (x) #Sets end to the end of the keyword
            return txt [end:end+30]
    

In [31]:
neew2['value'] = neew2['title'] .astype(str).apply(findAfter)

C:\Users\user\AppData\Local\Temp\ipykernel_22012\204853310.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neew2['value'] = neew2['title'] .astype(str).apply(findAfter)


In [32]:
 neew2['hotels_new'] = neew2['value'].str.extract('(.+?)-')

C:\Users\user\AppData\Local\Temp\ipykernel_22012\1613221675.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neew2['hotels_new'] = neew2['value'].str.extract('(.+?)-')


In [33]:
neew2['hotels_new']

3780         Plus C Mauritius 
3783       Chobe Safari Lodge 
3784     Mauricia Beachcomber 
3785               Rovos Rail 
3788                   Cocoon 
                 ...          
6793          Lagoon Attitude 
6795          Biyadhoo Island 
6803     Mauricia Beachcomber 
6804        South Palm Resort 
6805            My Blue Hotel 
Name: hotels_new, Length: 1114, dtype: object

In [34]:
neew3 = selected_rows.loc[selected_rows['domain']=='pentravel']

In [35]:
neew3['hotels_new'] = neew3['title']

C:\Users\user\AppData\Local\Temp\ipykernel_22012\2782757151.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neew3['hotels_new'] = neew3['title']


In [38]:
def findAfter2 (txt):
#len(txt.split(''))
    myList = txt.split(" ")
    for x in myList:
        if x=='4-star':
            start = txt.find (x) #Sets start to the start of keyword
            end = start + len (x) #Sets end to the end of the keyword
            return txt [end:end+30] 
        elif x=='3star':
            start = txt.find (x) #Sets start to the start of keyword
            end = start + len (x) #Sets end to the end of the keyword
            return txt [end:end+30] 
        elif x=='4star':
            start = txt.find (x) #Sets start to the start of keyword
            end = start + len (x) #Sets end to the end of the keyword
            return txt [end:end+30] 
        elif x=='3-star':
            start = txt.find (x) #Sets start to the start of keyword
            end = start + len (x) #Sets end to the end of the keyword
            return txt [end:end+30] 
        elif x=='45star':
            start = txt.find (x) #Sets start to the start of keyword
            end = start + len (x) #Sets end to the end of the keyword
            return txt [end:end+30] 
        elif x=='5star':
            start = txt.find (x) #Sets start to the start of keyword
            end = start + len (x) #Sets end to the end of the keyword
            return txt [end:end+30]
        
        elif x=='4,5-star':
            start = txt.find (x) #Sets start to the start of keyword
            end = start + len (x) #Sets end to the end of the keyword
            return txt [end:end+30]
        elif x=='5-star':
            start = txt.find (x) #Sets start to the start of keyword
            end = start + len (x) #Sets end to the end of the keyword
            return txt [end:end+30]

In [36]:
neew4 = selected_rows.loc[selected_rows['domain']=='flightcentre']

In [37]:
neew4.sample(10)

,scraped_url,agency_slug,country,title,description,inclusions,hotel_name,hotel_mapped,domain
87,https://www.flightcentre.co.za/product/14135671,flightcentre,mozambique,Azura Benguerra Island,\nYour Mozambique Holiday Package includes:\nR...,\nYour Mozambique Holiday Package includes:\n\...,NaN,NaN,flightcentre
55,https://www.flightcentre.co.za/product/14952266,flightcentre,united arab emirates,Abu Dhabi F1 Grand Prix 2022,\nYour Abu Dhabi Holiday Package includes:\n4 ...,\nYour Abu Dhabi Holiday Package includes:\n\n...,NaN,NaN,flightcentre
75,https://www.flightcentre.co.za/product/14135464,flightcentre,mozambique,Anantara Bazaruto Island Resort & Spa,\nYour Mozambique Holiday Package includes:\nR...,\nYour Mozambique Holiday Package includes:\n\...,NaN,NaN,flightcentre
29,https://www.flightcentre.co.za/product/15703260,flightcentre,tanzania,Sparkling Sunshine & Shady Palms,\nYour Zanzibar Holiday Package includes:\nRet...,\nYour Zanzibar Holiday Package includes:\n\nR...,NaN,NaN,flightcentre
37,https://www.flightcentre.co.za/product/14197139,flightcentre,maldives,Adaaran Club Rannalhi,\nYour Maldives Holiday Package includes:\nRet...,\nYour Maldives Holiday Package includes:\n\nR...,NaN,NaN,flightcentre
4,https://www.flightcentre.co.za/product/13250621,flightcentre,south africa,Kruger Park Splendour,\nYour Kruger National Park Holiday Package in...,\nYour Kruger National Park Holiday Package in...,NaN,NaN,flightcentre
23,https://www.flightcentre.co.za/product/15631761,flightcentre,canada,Tropical Paradise,\nYour Zanzibar Holiday Package includes:\nRet...,\nYour Zanzibar Holiday Package includes:\n\nR...,NaN,NaN,flightcentre
98,https://www.flightcentre.co.za/product/12616094,flightcentre,mauritius,Clear Skies & Turquoise Sea *JUNE/JULY SCHOOL ...,\nYour Mauritius Holiday Package includes:\nRe...,\nYour Mauritius Holiday Package includes:\n\n...,NaN,NaN,flightcentre
3,https://www.flightcentre.co.za/product/16323810,flightcentre,mauritius,Seas the Day in Mauritius,\nYour Mauritius Holiday Package includes:\nRe...,\nYour Mauritius Holiday Package includes:\n\n...,NaN,NaN,flightcentre
78,https://www.flightcentre.co.za/product/12615686,flightcentre,mauritius,Seas the Day in Mauritius,\nYour Mauritius Holiday Package includes:\nRe...,\nYour Mauritius Holiday Package includes:\n\n...,NaN,NaN,flightcentre


In [39]:
neew4['hotels_new'] = neew4['description'] .astype(str).apply(findAfter2)

C:\Users\user\AppData\Local\Temp\ipykernel_22012\1046999580.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neew4['hotels_new'] = neew4['description'] .astype(str).apply(findAfter2)


In [40]:
def first3(txx):
    lis=txx.split()[:3]
    new=' '.join(lis)
    return new

In [41]:
neew4['hotels_new']=neew4['hotels_new'] .astype(str).apply(first3)

C:\Users\user\AppData\Local\Temp\ipykernel_22012\31037954.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neew4['hotels_new']=neew4['hotels_new'] .astype(str).apply(first3)


In [ ]:
frames = [neew, neew1,neew2,neew4,neew3]
result = pd.concat(frames)

In [ ]:
display(result.info())

In [ ]:
nonselected_rows
frames3 = [nonselected_rows,result]
result3 = pd.concat(frames3)

In [ ]:
display(result3.info())

In [ ]:
arr = [
 'casterbridge-hollow',
 'elephant-plains-lodge',
 'idube-game-reserve',
 'marataba-trails-lodge',
 'hans-merensky-hotel-and-golf-estate',
 'bayethe-lodge',
 '4-day-fly-in-safari',
 'madikwe-hills-private-game-lodge',
 'phinda-homestead',
 'long-lee-manor',
 'camp-shawu',
 'thula-thula-private-game-reserve',
 'simbavati-river-lodge',
 'inn-on-the-square',
 'madikwe-safari-lodge',
 'country-boutique-hotel',
 'rhino-ridge-safari-lodge',
 'entabeni-lakeside-lodge',
 'kirkmans-kamp',
 'exeter-river-lodge',
 'ekuthuleni-lodge',
 'nedile-lodge',
 'leopard-hills-private-game-reserve',
 'thonga-beach-lodge',
 'nottens-bush-camp',
 'isibindi-zulu-lodge',
 'hillsnek-safari-camp',
 'camp-shonga',
 'leadwood-lodge',
 'nungubane-game-lodge',
 'impodimo-game-lodge',
 'alpine-heath-resort',
 'entabeni-kingfisher-lodge',
 'mandela-rhodes-place',
 'marataba-safari-lodge',
 'greenway-woods-resort-2980',
 'legend-golf-and-safari-resort',
 '2-day-fly-in-safari',
 'umkumbe-safari-lodge',
 'hollow-on-the-square-cape-town-city-hotel',
 'arathusa-safari-lodge',
 'perry-039-s-bridge-hollow-boutique-hotel',
 'sabi-sabi-bush-lodge',
 'hippo-hollow-country-estate',
 'oceana-beach-and-wildlife-resort',
 '4-day-camping-trip-to-kruger-national-park',
 'ecolux-boutique-hotel-and-spa',
 'bush-lodge',
 'londolozi-private-game-reserve',
 'nkomazi-game-reserve',
 '3-day-fly-in-safari',
 'little-bush-camp',
 'rhino-walking-safari',
 'kosi-forest-lodge',
 'white-elephant-safari-lodge',
 'jock-explorer-camp',
 'baradinckals-bush-lodge',
 'thandeka-game-lodge',
 'dulini-lodge',
 'leolapa',
 'mkuze-falls-private-game-reserve',
 'phinda-forest-lodge',
 'entabeni-wildside-safari-lodge',
 'entabeni-ravineside-lodge',
 'hanglip-mountain-lodge',
 'bushbreaks']

In [ ]:
#neew6 = df.loc[df['favorite_color'].isin(array)]

In [ ]:
neew6=selected_rows.loc[selected_rows['domain'].isin(arr)]

In [ ]:
neew6['hotels_new'] = neew6['title']

In [ ]:
frames4 = [neew6,result3]
result4 = pd.concat(frames4)

In [ ]:
display(result4.info())

In [ ]:
res=result4.sort_index()

In [ ]:
res.head(20)

In [ ]:
res.to_csv('hotels4.csv',index=False)

In [ ]:

res.to_excel("output.xlsx")